In [ ]:
import os

HOME = os.getcwd()
print(HOME)

## Download pre-trained weights

**NOTE:** YOLOv10 provides weight files pre-trained on the COCO dataset in various sizes. Let's download them.

yolov10n -> https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt  
yolov10s -> https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt  
yolov10m -> https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt  
yolov10b -> https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt  
yolov10x -> https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt  
yolov10l -> https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt  

## Inference with Pre-trained COCO Model

**NOTE:** YOLOv10 is based on YOLOv8, and like YOLOv8, it can be used in both CLI and SDK modes.

### 🐍 Python SDK

In [ ]:
from ultralytics import YOLOv10

model = YOLOv10(f'{HOME}/weights/yolov10n.pt')
results = model(source=f'{HOME}/data/dog.jpeg', conf=0.25)

print(results[0].boxes.xyxy)
print(results[0].boxes.conf)
print(results[0].boxes.cls)

## Download dataset from Roboflow Universe

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="your roboflow API")
project = rf.workspace("ilchrees").project("gigadataset")
version = project.version(5)
dataset = version.download("yolov8")

**NOTE:**
Make sure the last 4 lines of the data.yaml file have the following format:

```
test: ../test/images
train: ../train/images
val: ../valid/images
```

If using a dataset from Roboflow Universe, run the command below. 👇🏻

## Custom Training

In [ ]:
from ultralytics import YOLOv10


# If you want to finetune the model with pretrained weights, you could load the 
# pretrained weights like below

# model = YOLOv10.from_pretrained('jameslahm/yolov10{n/s/m/b/l/x}')
# or
# model = YOLOv10('yolov10{n/s/m/b/l/x}.pt')
# else
# model = YOLOv10()


model = YOLOv10("yolov10s.pt")
model.train(data=f"{dataset.location}/data.yaml", epochs=100, batch=32, imgsz=640)

## Inference with Custom Model

**NOTE:** Let's start by loading our newly trained model.

In [ ]:
from ultralytics import YOLOv10
import supervision as sv 
import random

from Utils.annotate import YOLO_annotateImage, YOLO_annotateVideo


model = YOLOv10(f'{HOME}/runs/detect/train/weights/best.pt')

dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{dataset.location}/valid/images",
    annotations_directory_path=f"{dataset.location}/valid/labels",
    data_yaml_path=f"{dataset.location}/data.yaml"
)

random_image = random.choice(list(dataset.images.keys()))
random_image = dataset.images[random_image]

YOLO_annotateImage(model, random_image)

YOLO_annotateVideo(model, video_path="DataSources\\Videos\\TAMU_vs_VT_1.mov")